#Import libraries

In [4]:
import numpy as np
import struct

#Loading IDX files

In [5]:
def load_images(path, limit):
    with open(path, 'rb') as f:
        magic, num, rows, cols = struct.unpack(">IIII", f.read(16))
        images = np.frombuffer(f.read(), dtype=np.uint8)
        images = images.reshape(num, rows, cols)
        return images[:limit] / 255.0

def load_labels(path, limit):
    with open(path, 'rb') as f:
        magic, num = struct.unpack(">II", f.read(8))
        labels = np.frombuffer(f.read(), dtype=np.uint8)
        return labels[:limit]

#Loading the dataset

In [6]:
# Training Data (500)
X_train = load_images("train-images-idx3-ubyte", 500)
y_train = load_labels("train-labels-idx1-ubyte", 500)

# Load test data
X_test = load_images("t10k-images-idx3-ubyte", 100)
y_test = load_labels("t10k-labels-idx1-ubyte", 100)

In [7]:
print("Training shape:", X_train.shape)
print("Testing shape:", X_test.shape)

Training shape: (500, 28, 28)
Testing shape: (100, 28, 28)


# INITIALIZE PARAMETERS

In [8]:
num_filters = 12
filter_size = 3
num_classes = 10
learning_rate = 0.005

filters = np.random.randn(num_filters, filter_size, filter_size) * 0.1
fc_input_size = 13 * 13 * num_filters
W_fc = np.random.randn(fc_input_size, num_classes) * 0.1
b_fc = np.zeros(num_classes)

# ACTIVATION & LOSS FUNCTIONS

In [9]:
def relu(x):
    return np.maximum(0, x)

def softmax(x):
    exp = np.exp(x - np.max(x))
    return exp / np.sum(exp)

def cross_entropy(pred, label):
    return -np.log(pred[label] + 1e-9)

# CONVOLUTION

In [10]:
def conv(image, filters):
    h, w = image.shape
    f = filters.shape[1]
    output = np.zeros((num_filters, h-f+1, w-f+1))

    for n in range(num_filters):
        for i in range(h-f+1):
            for j in range(w-f+1):
                output[n, i, j] = np.sum(
                    image[i:i+f, j:j+f] * filters[n]
                )
    return output

# MAX POOLING (2x2)

In [11]:
def maxpool(feature_map):
    num_f, h, w = feature_map.shape
    output = np.zeros((num_f, h//2, w//2))

    for n in range(num_f):
        for i in range(0, h, 2):
            for j in range(0, w, 2):
                output[n, i//2, j//2] = np.max(
                    feature_map[n, i:i+2, j:j+2]
                )
    return output

# TRAINING

In [14]:
for epoch in range(50):
    total_loss = 0
    correct = 0

    for i in range(500):
        image = X_train[i]
        label = y_train[i]

        # Forward pass
        conv_out = conv(image, filters)
        relu_out = relu(conv_out)
        pool_out = maxpool(relu_out)

        flat = pool_out.flatten()
        logits = np.dot(flat, W_fc) + b_fc
        probs = softmax(logits)

        loss = cross_entropy(probs, label)
        total_loss += loss

        if np.argmax(probs) == label:
            correct += 1

        # Backprop (Fully connected layer only)
        d_logits = probs
        d_logits[label] -= 1

        dW_fc = np.outer(flat, d_logits)
        db_fc = d_logits

        W_fc -= learning_rate * dW_fc
        b_fc -= learning_rate * db_fc

    print("Epoch:", epoch+1,
          "Loss:", total_loss/500,
          "Train Accuracy:", correct/500)

Epoch: 1 Loss: 0.4579734168700988 Train Accuracy: 0.876
Epoch: 2 Loss: 0.44994552107852476 Train Accuracy: 0.884
Epoch: 3 Loss: 0.44546752205735873 Train Accuracy: 0.89
Epoch: 4 Loss: 0.44099529943144494 Train Accuracy: 0.89
Epoch: 5 Loss: 0.4366769816291712 Train Accuracy: 0.89
Epoch: 6 Loss: 0.43250469639462213 Train Accuracy: 0.89
Epoch: 7 Loss: 0.4284602320583696 Train Accuracy: 0.892
Epoch: 8 Loss: 0.42452934065631037 Train Accuracy: 0.894
Epoch: 9 Loss: 0.42070130355676255 Train Accuracy: 0.894
Epoch: 10 Loss: 0.4169678147046128 Train Accuracy: 0.894
Epoch: 11 Loss: 0.41332221822209053 Train Accuracy: 0.896
Epoch: 12 Loss: 0.40975902073356313 Train Accuracy: 0.896
Epoch: 13 Loss: 0.4062735727808641 Train Accuracy: 0.896
Epoch: 14 Loss: 0.4028618545136318 Train Accuracy: 0.898
Epoch: 15 Loss: 0.3995203277425251 Train Accuracy: 0.902
Epoch: 16 Loss: 0.39624583154541065 Train Accuracy: 0.902
Epoch: 17 Loss: 0.3930355071823889 Train Accuracy: 0.902
Epoch: 18 Loss: 0.3898867431297876 

#Testing

In [15]:
correct = 0

for i in range(100):
    image = X_test[i]
    label = y_test[i]

    conv_out = conv(image, filters)
    relu_out = relu(conv_out)
    pool_out = maxpool(relu_out)

    flat = pool_out.flatten()
    logits = np.dot(flat, W_fc) + b_fc
    probs = softmax(logits)

    if np.argmax(probs) == label:
        correct += 1

print("Test Accuracy:", correct/100)

Test Accuracy: 0.77


#Interpretation
The loss decreased steadily across epochs, indicating that the model was learning effectively. Training accuracy improved from 27% to 70%, demonstrating that the CNN was successfully extracting meaningful features. The final test accuracy of 65% shows that the model generalizes reasonably well to unseen data.

After tuning the learning rate, increasing the number of filters, and training for more epochs, the model achieved a test accuracy of 77%, showing improved feature extraction and better generalization